# Stacks and Queues

## 3.1 Three in One
Describe how you could you use a single array to implement three stacks.

### Solution
#### Approach1: Fixed Division
・すべてのスタックが同じ長さの場合、リストを3分割する考え方  
  For stack1: リストのIndex [0, n/3)  
  For stack1: リストのIndex [n/3, 2n/3)  
  For stack1: リストのIndex [2n/3, n)  

In [1]:
class FixedMultiStack:
    def __init__(self):
        self.numberOfStacks = 3

## 3.2 Stack Min
How would you design a stack which, in addition to push and pop, has a function min which returns the minimum element?
Push, pop and min should all operate in O(1) time.

In [4]:
class Node:
    def __init__(self, data, nextNode=None):
        self.data = data          # Nodeに格納する実際のデータが入る
        self.next = nextNode # Nodeクラスのインスタンスが入る
        

class StackMin:
    def __init__(self):
        self.top = None
        self.min = float('inf')
    
    def push(self, item):
        # top がNoneなら、ノードのnextはNone, topがあればノードのnextはtopの内容
        self.top = Node(item, None if self.top == None else self.top)
        if self.min is not None: #本当は不要だけど、MyQueueがこのクラスを利用するときにこのIFが必要になる
            self.min = min(self.min, self.top.data)
    
    def pop(self):
        # stackの最小値のデータを更新
        if self.top.data == self.min and self.top.next != None:
            self.__updateMin() 
        elif self.top.next == None: # ノードが最後の一個の場合
            self.min = None
        
        # returnするNode
        popNode = self.top 
        self.top = self.top.next # topを入れ替え
        return popNode.data
    
    # minは予約語なのでmin_にしておく
    def min_(self):
        return self.min
    
    def peek(self):
        return self.top.data 
    
    def isEmpty(self):
        return True if self.top == None else False

    def __updateMin(self):
        self.min = float("inf")
        currentNode = self.top.next # self.topは消されるので、self.top.nextから見ていく
        while True:
            self.min = min(self.min, currentNode.data)
            if currentNode.next == None:
                break
            currentNode = currentNode.next

In [3]:
#  基本的なstackの動作を確認
stack = StackMin()
stack.push(0)
stack.push(1)
stack.push(2)
print("peek (expect:2 ) : ", stack.peek())
stack.pop()
stack.pop()
print("show min(expect: 0 ):  ", str(stack.min_())) # minを表示
stack.pop()
print("isEmpty (expect: True)" , stack.isEmpty())

peek (expect:2 ) :  2
show min(expect: 0 ):   0
isEmpty (expect: True) True


## 3.3 Stack of Plates
言葉の通り、「プレートのスタック」をイメージする。あまり高すぎると、倒れるかもしれない。  
したがって、現実には、前のスタックがある閾値を超えたら、新しいスタックを開始することが多いでしょう。

"SetOfStacks "は複数のスタックから構成され、前のスタックが容量を超えたら新しいスタックを作成する必要があるはずです。

[FOLLOW UP]
特定のサブスタックに対して pop 操作を行う関数 "popAt" を実装する。

In [4]:
class SetOfStacks:
    def __init__(self, threshold=3):
        self.threshold = threshold
        self.currentStack = None
        self.currentStackLength = 0
        self.preStack = None
    
    def push(self, item):
        if self.currentStack == None:
            self.currentStack = StackMin()
            self.currentStack.push(item)
            self.currentStackLength += 1
            return 0
        
        # 閾値に達する場合
        if self.currentStackLength == self.threshold :
            nextStack = StackMin()
            self.preStack = Node(self.currentStack, nextStack)
            self.currentStack = nextStack
            self.currentStack.push(item)
            self.currentStackLength = 1
        else:
            self.currentStack.push(item)
            self.currentStackLength += 1
    
        return 0

    def pop(self):
        # 現在のstackの長さが1(現在のstackでpopするものがない場合)の場合、前のstackから持ってくる
        if self.currentStackLength == 1:
            # stackを入れ替えるだけでpop()は完了するので、self.currentStack.pop()は呼び出さない
            self.currentStack = self.preStack.data 
            self.preStack = self.preStack.next
            self.currentStackLength = self.threshold
            
        else:
            self.currentStack.pop()
            self.currentStackLength -= 1
        
        return 0
    
    def peek(self):
            return self.currentStack.peek()

In [5]:
setOfStacks = SetOfStacks(2)
setOfStacks.push(0)
setOfStacks.push(1)
setOfStacks.push(10)
setOfStacks.push(11)
print("peek(expect: 11) : ", setOfStacks.peek())
setOfStacks.pop()
print("peek(expect: 10) : ", setOfStacks.peek())
setOfStacks.pop()
print("peek(expect: 1) : ", setOfStacks.peek())
setOfStacks.pop()
print("peek(expect: 0) : ", setOfStacks.peek())

peek(expect: 11) :  11
peek(expect: 10) :  10
peek(expect: 1) :  1
peek(expect: 0) :  0


In [6]:
setOfStacks = SetOfStacks(2)
setOfStacks.push(0)
setOfStacks.push(1)
setOfStacks.push(10)
setOfStacks.push(11)
setOfStacks.push(100)
setOfStacks.push(101)
setOfStacks.pop()
setOfStacks.pop()
print("peek(expect: 11) : ", setOfStacks.peek())

peek(expect: 11) :  11


## 3.4 Queue via Stacks
Implement a MyQueue class which implements a queue using two stacks.  
■ addの時  
pop用のstackに入っているNodeを全てpush用のstackに移してからaddする  

■ removeの時  
push用のstackに入っているNodeを全てpush用のstackに移してからremoveする

In [19]:
class MyQueue:
    def __init__(self):
        self.forAddStack = StackMin()
        self.forRemoveStack = StackMin()
        self.peek = -float('inf')
    
    def add(self, value):
        # self.forRemoveStackの中身を全てself.forAddStackに移す
        self.migration(self.forRemoveStack, self.forAddStack)
        
        # self.forAddStackにaddする
        self.forAddStack.push(value)
        
    
    def remove(self):
        # self.forAddStackの中身を全てself.forRemoveStackに移す
        self.migration(self.forAddStack, self.forRemoveStack)
        
        # self.forRemoveStackからremoveする
        return self.forRemoveStack.pop()
        
    
    def peek(self):
        pass
    
    def isEmpty(self):
        return self.forAddStack.isEmpty() or self.forRemoveStack.isEmpty()
        
    # move from stack1 to stack2
    def migration(self, stack1, stack2):
        while stack1.isEmpty() != True:
            popNode = stack1.pop()
            stack2.push(popNode)


In [22]:
queue = MyQueue()
queue.add(1)
queue.add(2)
queue.add(3)
queue.add(4)
print("===========")
print('remove(expect : 1) : ', queue.remove())
print("===========")
print('remove(expect : 2): ', queue.remove())
print("===========")
print('remove(expect : 3): ', queue.remove())
print("===========")
queue.add(5)
print('remove(expect : 4): ', queue.remove())
print("===========")
print('remove(expect : 5): ', queue.remove())
print("===========")

remove(expect : 1) :  1
remove(expect : 2):  2
remove(expect : 3):  3
remove(expect : 4):  4
remove(expect : 5):  5


## 3.5 Sort Stack

Write a program to sort a stack such that the smallest items are on the top.   
You can use an additional temporary stack, but you may not copy the elements into any other data structure(such as an array).  
The stack supports the following operations: push, pop, peek and isEmpty.

HINT1(#14):  
ソートの一つの方法としては、配列を順番に見て行って、新しい配列にソートされた順序でインサートすること。  
これがstackでできる?  


In [7]:
# O(N^2) time  , O(N) space
def sortStack(unsortedStack):
    sortedStack = StackMin() # 普通のStackクラスとして利用
    while unsortedStack.isEmpty() != True:
        tmp = unsortedStack.pop()
        # tmpがpushできるpositionになるまで、sortedStackのノードをunsortedStackにpushし続ける
        while sortedStack.isEmpty() != True and sortedStack.peek() > tmp:
            unsortedStack.push(sortedStack.pop())
        sortedStack.push(tmp)
    return sortedStack

unsortedStack = StackMin()
unsortedStack.push(1)
unsortedStack.push(5)
unsortedStack.push(3)
unsortedStack.push(5)
unsortedStack.push(9)
unsortedStack.push(3)
sortedStack = sortStack(unsortedStack) # sort

print('====== Show sortedStack ======')
while sortedStack.isEmpty() != True :
    print(sortedStack.pop())

====== Show sortedStack ======
9
5
5
3
3
1


## 3.6 Animal Shelter
An animal shelter, which holds only dogs and cats, operates on a strictly(厳密に) "first in, first out" basis.  
People must adopt either the "oldest"(based on arrival time) of all animals at the shelter, or they can select whether they would prefer a dog or a cat (and will receive the oldest animal of that type). They cannot select which specific animal they would like. Create the data structures to maintain this system and implement operations such as `enqueue, dequeuAny, dequeueDog, and dequeueCat`. You may use the built-in LinkedList data stucture.

In [17]:
class Node:
    def __init__(self, value):
        self.value = value
        self.next = None

class LinkedList:
    def __init__(self):
        self.head = None
        self.rear = None
    
    # 前から追加
    def insert(self, value):
        newNode = Node(value)
        # 最初の一つ目
        if self.head == None:
            self.head = newNode
            self.rear = newNode
            return
        
        self.rear.next = newNode
        self.rear = self.rear.next
    
    # 前から削除
    def delete(self):
        if self.head == self.rear:
            tmp = self.head
            self.head = None
            self.rear = None
        else:
            tmp = self.head
            self.head = self.head.next
        return tmp.value
    
    # 今回はqueueとして扱えるように、peekではhead.valueを返す
    # (stackの場合は逆のrear.valueを返す)
    def peek(self):
        return self.head.value
        
    def isEmpty(self):
        return True if self.head == None and self.rear == None else False

class Animal:
    def __init__(self, name):
        self.order = None
        self.name = name
    
class AnimalShelter:
    def __init__(self):
        self.dogs = LinkedList()
        self.cats = LinkedList()
        self.order = 0
    
    def enqueue(self, animal):
        animal.order = self.order
        self.order += 1
        
        if animal.name == 'dog':
            self.dogs.insert(animal)
        elif animal.name == 'cat':
            self.cats.insert(animal)
    
    def dequeuAny(self):
        if self.dogs.isEmpty() == True:
            return self.dequeuCat()
        elif self.cats.isEmpty() == True:
            return self.dequeuDog()
        
        dog = self.dogs.peek()
        cat = self.cats.peek()
        if dog.order < cat.order:
            return self.dequeuDog()
        else:
            return self.dequeuCat()
        
    def dequeuCat(self):
        return self.cats.delete().name
        
    def dequeuDog(self):
        return self.dogs.delete().name


animals = [Animal('dog'), Animal('cat'), Animal('dog'), Animal('dog'), Animal('cat'), Animal('cat')]
animalShelter = AnimalShelter()
for animal in animals:
    animalShelter.enqueue(animal)

print('dequeuAny(expect: dog) : ', animalShelter.dequeuAny())
print('dequeuAny(expect: cat) : ', animalShelter.dequeuAny())
print('dequeuAny(expect: dog) : ', animalShelter.dequeuAny())
print('dequeuAny(expect: dog) : ', animalShelter.dequeuAny())
print('dequeuAny(expect: cat) : ', animalShelter.dequeuAny())
print('dequeuAny(expect: cat) : ', animalShelter.dequeuAny())

print('======================')

animals = [Animal('dog'), Animal('cat'), Animal('dog'), Animal('dog'), Animal('cat'), Animal('cat')]
animalShelter = AnimalShelter()
for animal in animals:
    animalShelter.enqueue(animal)
    
print('dequeuAny(expect: cat) : ', animalShelter.dequeuCat())
print('dequeuAny(expect: dog) : ', animalShelter.dequeuDog())
print('dequeuAny(expect: cat) : ', animalShelter.dequeuCat())
print('dequeuAny(expect: cat) : ', animalShelter.dequeuCat())
print('dequeuAny(expect: dog) : ', animalShelter.dequeuAny())
print('dequeuAny(expect: dog) : ', animalShelter.dequeuAny())


dequeuAny(expect: dog) :  dog
dequeuAny(expect: cat) :  cat
dequeuAny(expect: dog) :  dog
dequeuAny(expect: dog) :  dog
dequeuAny(expect: cat) :  cat
dequeuAny(expect: cat) :  cat
dequeuAny(expect: cat) :  cat
dequeuAny(expect: dog) :  dog
dequeuAny(expect: cat) :  cat
dequeuAny(expect: cat) :  cat
dequeuAny(expect: dog) :  dog
dequeuAny(expect: dog) :  dog
